In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [3]:
#依赖的包
from lib.dataloader import normal_and_generate_dataset_time, get_mask, get_adjacent, get_grid_node_map_maxtrix
#攻击函数import
from lib.utils_new import mask_loss, compute_loss, predict_and_evaluate, attack, random_attack, fgsm_attack, min_attack
from lib.utils_new import saliency, saliency_loss, attack_js, attack_js,attack_saliency,rand_attack
from model.GSNet import GSNet
from lib.early_stop import EarlyStopping
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import json
import configparser
import pickle as pkl
from time import time
from datetime import datetime
import shutil
import argparse
import random
import math

import sys
import os


/home/wyb/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wyb/miniconda3/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
#完整运行时会生成txt日志记录#
train_log_filename = "test_log_ch.txt"
train_log_filepath = os.path.join("./", train_log_filename)
now = datetime.now()
date_time = now.strftime("|%Y-%m-%d, %H:%M:%S| ")

curPath = os.path.abspath(os.path.dirname('__file__'))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)

In [5]:
#配置文件载入，初始参数记录
#NYC
from mmcv import Config
cfgin = Config.fromfile('lib/attack_all_setting.yaml')
#######


In [6]:
if cfgin.dataset =='NYC':
    config_filename = 'config/nyc/GSNet_NYC_Config.json'
else: 
    config_filename = "config/chicago/GSNet_Chicago_Config.json"
with open(config_filename, 'r') as f:
    config = json.loads(f.read())
# 写入测试超参数
with open(train_log_filepath, "a") as f:
    f.write('测试文件运行时间：'+date_time)
    f.write('\r\n')
    f.write('超参数设置：')
    f.write('\r\n')
    f.write(json.dumps(config, sort_keys=True, indent=4))
    f.write('\r\n')
f.close
#GPU设置
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

#文件读取
#命令行参数实质上就是赋初始值的作用，可以删除直接赋值就好
north_south_map = config['north_south_map']
west_east_map = config['west_east_map']

all_data_filename = config['all_data_filename']
mask_filename = config['mask_filename']

road_adj_filename = config['road_adj_filename']
risk_adj_filename = config['risk_adj_filename']
poi_adj_filename = config['poi_adj_filename']
grid_node_filename = config['grid_node_filename']
grid_node_map = get_grid_node_map_maxtrix(grid_node_filename)
num_of_vertices = grid_node_map.shape[1]


patience = config['patience']#用处：
delta = config['delta']

if config['seed'] is not None:
    seed = config['seed']
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


train_rate = config['train_rate']
valid_rate = config['valid_rate']

recent_prior = config['recent_prior']
week_prior = config['week_prior']
one_day_period = config['one_day_period']
days_of_week = config['days_of_week']
pre_len = config['pre_len']
seq_len = recent_prior + week_prior

training_epoch = config['training_epoch']


True


# 原始数据处理
all_data.pkl       (8760, 48, 20, 20) 8760=365*24
grid_node_map.pkl  (400, 243)  映射矩阵
输出特征大小
训练集：4584*7*48*20*20
验证集：1080*7*48*20*20
测试集：1080*7*48*20*20
7的含义：t =673时刻是，输入7个时间片，[1, 169, 337, 505, 670, 671, 672]

In [14]:
batch_size = 32
learning_rate = config['learning_rate']

#模型结构
num_of_gru_layers = config['num_of_gru_layers']
gru_hidden_size = config['gru_hidden_size']
gcn_num_filter = config['gcn_num_filter']

loaders = []
scaler = ""
train_data_shape = ""
graph_feature_shape = ""
   ################## 获取测试数据#########################
for idx, (x, y, target_times, high_x, high_y, high_target_times, scaler) in enumerate(normal_and_generate_dataset_time(
        all_data_filename,
        train_rate=train_rate,
        valid_rate=valid_rate,
        recent_prior=recent_prior,
        week_prior=week_prior,
        one_day_period=one_day_period,
        days_of_week=days_of_week,
        pre_len=pre_len)):
    if False:#前100个，没写默认是false
        x = x[:100]
        y = y[:100]
        target_times = target_times[:100]
        high_x = high_x[:100]
        high_y = high_y[:100]
        high_target_times = high_target_times[:100]

    if 'nyc' in all_data_filename:
        graph_x = x[:, :, [0, 46, 47], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))#4584*7*3*400,注意reshape和交换维度的不同
        high_graph_x = high_x[:, :, [0, 46, 47], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))#1337*7*3*400
        graph_x = np.dot(graph_x, grid_node_map)#4584*7*3*243
        high_graph_x = np.dot(high_graph_x, grid_node_map)
    if 'chicago' in all_data_filename:
        graph_x = x[:, :, [0, 39, 40], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))
        high_graph_x = high_x[:, :, [0, 39, 40], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))
        graph_x = np.dot(graph_x, grid_node_map)
        high_graph_x = np.dot(high_graph_x, grid_node_map)

    print("feature:", str(x.shape), "label:", str(y.shape), "time:", str(target_times.shape),
            "high feature:", str(high_x.shape), "high label:", str(high_y.shape))
    print("graph_x:", str(graph_x.shape),
            "high_graph_x:", str(high_graph_x.shape))
    if x.shape[0] ==1080:
        b = np.ones_like(x)
        b = np.where(x>1,1,0)

    if idx == 0:
        scaler = scaler
        train_data_shape = x.shape
        time_shape = target_times.shape
        graph_feature_shape = graph_x.shape
    loaders.append(Data.DataLoader(
        Data.TensorDataset(
            torch.from_numpy(x),
            torch.from_numpy(target_times),#4584*32
            torch.from_numpy(graph_x),
            torch.from_numpy(y)
        ),
        batch_size=batch_size,
        shuffle=(idx == 0)
    ))
    if idx == 2:
        high_test_loader = Data.DataLoader(
            Data.TensorDataset(
                torch.from_numpy(high_x),
                torch.from_numpy(high_target_times),
                torch.from_numpy(high_graph_x),
                torch.from_numpy(high_y)
            ),
            batch_size=batch_size,
            shuffle=(idx == 0)
        )
train_loader, val_loader, test_loader = loaders
################ 获取测试数据##################
nums_of_filter = []
for _ in range(2):
    nums_of_filter.append(gcn_num_filter)

GSNet_Model = GSNet(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
                    gru_hidden_size, time_shape[1], graph_feature_shape[2],
                    nums_of_filter, north_south_map, west_east_map)
print("模型参数---------------------------------------------")
print(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
        gru_hidden_size, time_shape[1], graph_feature_shape[2],
        nums_of_filter)

# multi gpu
if torch.cuda.device_count() > 10:
    print("Let's use", torch.cuda.device_count(), "GPUs!", flush=True)
    GSNet_Model = nn.DataParallel(GSNet_Model)
############### 载入模型######################
GSNet_Model.to(device)
# print(GSNet_Model)
GSNet_Model.load_state_dict(torch.load("data/model_35.pt"))
print('数据集为：'+cfgin.dataset)
print("模型加载成功！")

num_of_parameters = 0
for name, parameters in GSNet_Model.named_parameters():
    num_of_parameters += np.prod(parameters.shape)
print("Number of Parameters: {}".format(num_of_parameters), flush=True)

trainer = optim.Adam(GSNet_Model.parameters(), lr=learning_rate)
early_stop = EarlyStopping(patience=patience, delta=delta)

risk_mask = get_mask(mask_filename)
road_adj = get_adjacent(road_adj_filename)
risk_adj = get_adjacent(risk_adj_filename)
if poi_adj_filename == "":
    poi_adj = None
else:
    poi_adj = get_adjacent(poi_adj_filename)


feature: (2827, 7, 41, 20, 20) label: (2827, 1, 20, 20) time: (2827, 32) high feature: (826, 7, 41, 20, 20) high label: (826, 1, 20, 20)
graph_x: (2827, 7, 3, 197) high_graph_x: (826, 7, 3, 197)
feature: (494, 7, 41, 20, 20) label: (494, 1, 20, 20) time: (494, 32) high feature: (143, 7, 41, 20, 20) high label: (143, 1, 20, 20)
graph_x: (494, 7, 3, 197) high_graph_x: (143, 7, 3, 197)
feature: (495, 7, 41, 20, 20) label: (495, 1, 20, 20) time: (495, 32) high feature: (144, 7, 41, 20, 20) high label: (144, 1, 20, 20)
graph_x: (495, 7, 3, 197) high_graph_x: (144, 7, 3, 197)
模型参数---------------------------------------------
41 6 7 1 128 32 3 [64, 64]
数据集为：Chicago
模型加载成功！
Number of Parameters: 3756715


In [15]:
#配置log日志输出
from lib.myfunction import get_root_logger, logger_info
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")


2023-05-17 15:07:46,680 - INFO - Loading config file from 


In [26]:
torch.cuda.empty_cache()

In [27]:
#攻击，并写入数据,20230315导入包修改为lib.utils_new_round
import yaml
from mmcv import Config
from lib.myfunction import log_test_results
from lib.outzoom import log_test_csv,node_map
from lib.outzoom import STZINB,min_impr,pgd_impr,random_impr,log_test_csv_time_K
from lib.myfunction import get_root_logger, logger_info
#配置logo
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")

#根据配置选择攻击方法
if cfgin.attacker == 'white_attacker':
    header = ['time', 'epoch', 'dataset', 'model', 'node_select', 'method', 'K', 'batch size',
              'clean_RMSE', 'clean_recall', 'clean_MAP', 'clean_RCR',
              'adv_RMSE', 'adv_recall', 'adv_MAP', 'adv_RCR',
              'local_adv_RMSE', 'local_adv_recall', 'local_adv_MAP', 'local_adv_RCR',
              '00']
    #文件名
    file_name = 'result_chicago-data-{}_num-nodes{}_eps{}-model-{}'.format(
        cfgin.dataset, cfgin.test_attack_nodes, cfgin.test_epsilon, cfgin.backbone)
    log_test_results(cfgin.result_dir, header, file_name)
    #可能的时间循环
    #攻击方法+节点选择组合
    for epoch in range(1):
        #break
        for out_K in cfgin.K:
            for i in cfgin.select_node:
                for j in cfgin.attack_method:
                    logger.info(i+'_'+j+"_"+str(out_K))
                    parm_att = j  # 攻击方法的函数名称
                    parm_node = i  # 节点选择的方法
                    # 进行攻击,输出攻击日志
                    # 1. 节点选择，返回攻击的节点
                    ack_map = node_map(i,12,out_K, 5,cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                       grid_node_map, device, data_type='nyc')
                    # 2. 攻击方法选择，返回未攻击后的预测值，标签值,攻击前的预测值，攻击过程打印每个信息
                    adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                            grid_node_map, scaler, risk_mask, device, data_type='nyc')
                    # 计算结果并打印、存储,注意表头信息需要传递参数
                    log_test_csv_time_K(epoch, out_K, adv_val_predict, val_target,
                                        val_predict, cfgin, i, j, file_name, risk_mask)
                    torch.cuda.empty_cache()
    for epoch in range(1):
        #break
        for out_K in [40]:
            for a in range(12):
                i = 'HGNS'
                j = 'STZINB'
                logger.info(i+'_'+j+"_"+str(out_K))
                parm_att = j  # 攻击方法的函数名称
                parm_node = i  # 节点选择的方法
                # 进行攻击,输出攻击日志
                # 1. 节点选择，返回攻击的节点
                ack_map = node_map(i, a, out_K, 5, cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                    grid_node_map, device, data_type='chicago')
                # 2. 攻击方法选择，返回未攻击后的预测值，标签值,攻击前的预测值，攻击过程打印每个信息
                adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                        grid_node_map, scaler, risk_mask, device, data_type='nyc')
                # 计算结果并打印、存储,注意表头信息需要传递参数
                log_test_csv_time_K(epoch, out_K, adv_val_predict, val_target,
                                    val_predict, cfgin, i, j, file_name, risk_mask)
                torch.cuda.empty_cache()


2023-05-17 15:26:41,278 - INFO - Loading config file from 
2023-05-17 15:26:41,281 - INFO - Centrality_min_impr_10


time,epoch,dataset,model,node_select,method,K,batch size,clean_RMSE,clean_recall,clean_MAP,clean_RCR,adv_RMSE,adv_recall,adv_MAP,adv_RCR,local_adv_RMSE,local_adv_recall,local_adv_MAP,local_adv_RCR,00


2023-05-17 15:26:43,870 - INFO - Info:  [64/495 (12%)]	  clean_RMSE: 0.5409 clean_Recall: 20.9524 clean_MAP: 0.0687 clean_RCR: 0.8350   adv_RMSE: 0.5428 adv_Recall: 21.9048 adv_MAP: 0.0697 adv_RCR: 0.8267 time:1.165
2023-05-17 15:26:46,337 - INFO - Info:  [128/495 (25%)]	  clean_RMSE: 0.5505 clean_Recall: 17.7083 clean_MAP: 0.1596 clean_RCR: 0.8461   adv_RMSE: 0.5493 adv_Recall: 21.8750 adv_MAP: 0.1780 adv_RCR: 0.8186 time:1.221
2023-05-17 15:26:48,795 - INFO - Info:  [192/495 (38%)]	  clean_RMSE: 0.5346 clean_Recall: 20.6186 clean_MAP: 0.0620 clean_RCR: 0.8615   adv_RMSE: 0.5356 adv_Recall: 18.5567 adv_MAP: 0.0582 adv_RCR: 0.8708 time:1.205
2023-05-17 15:26:51,185 - INFO - Info:  [256/495 (50%)]	  clean_RMSE: 0.5212 clean_Recall: 16.4948 clean_MAP: 0.1009 clean_RCR: 0.8825   adv_RMSE: 0.5215 adv_Recall: 16.4948 adv_MAP: 0.0972 adv_RCR: 0.8826 time:1.198
2023-05-17 15:26:53,640 - INFO - Info:  [320/495 (62%)]	  clean_RMSE: 0.5701 clean_Recall: 18.2540 clean_MAP: 0.0790 clean_RCR: 0.859

2023-05-17 15:27:01,0,Chicago,Gsnet,Centrality,min_impr,10,32,0.5417841608881183,20.63208109719738,0.08756782175523166,0.8295174198583164,0.5434172802233007,20.155038759689923,0.08659833020097298,0.8331832641717047,0.060965836896136764,98.89424675756568,0.9869665956528383,0.0028277937386682363


2023-05-17 15:27:05,848 - INFO - Info:  [64/1080 (12%)]	  clean_RMSE: 0.5409 clean_Recall: 20.9524 clean_MAP: 0.0687 clean_RCR: 0.8350   adv_RMSE: 0.5432 adv_Recall: 21.9048 adv_MAP: 0.0694 adv_RCR: 0.8273 time:1.287
2023-05-17 15:27:08,541 - INFO - Info:  [128/1080 (25%)]	  clean_RMSE: 0.5505 clean_Recall: 17.7083 clean_MAP: 0.1596 clean_RCR: 0.8461   adv_RMSE: 0.5495 adv_Recall: 22.9167 adv_MAP: 0.1803 adv_RCR: 0.7975 time:1.334
2023-05-17 15:27:11,223 - INFO - Info:  [192/1080 (38%)]	  clean_RMSE: 0.5346 clean_Recall: 20.6186 clean_MAP: 0.0620 clean_RCR: 0.8615   adv_RMSE: 0.5354 adv_Recall: 18.5567 adv_MAP: 0.0576 adv_RCR: 0.8707 time:1.315
2023-05-17 15:27:13,836 - INFO - Info:  [256/1080 (50%)]	  clean_RMSE: 0.5212 clean_Recall: 16.4948 clean_MAP: 0.1009 clean_RCR: 0.8825   adv_RMSE: 0.5211 adv_Recall: 15.4639 adv_MAP: 0.0668 adv_RCR: 0.9045 time:1.316
2023-05-17 15:27:16,527 - INFO - Info:  [320/1080 (62%)]	  clean_RMSE: 0.5701 clean_Recall: 18.2540 clean_MAP: 0.0790 clean_RCR: 

...

2023-05-17 15:28:59,467 - INFO - Info:  [64/495 (12%)]	  clean_RMSE: 0.5409 clean_Recall: 20.9524 clean_MAP: 0.0687 clean_RCR: 0.8350   adv_RMSE: 0.5427 adv_Recall: 21.9048 adv_MAP: 0.0760 adv_RCR: 0.8259 time:1.172
2023-05-17 15:29:01,955 - INFO - Info:  [128/495 (25%)]	  clean_RMSE: 0.5505 clean_Recall: 17.7083 clean_MAP: 0.1596 clean_RCR: 0.8461   adv_RMSE: 0.5498 adv_Recall: 19.7917 adv_MAP: 0.1583 adv_RCR: 0.8317 time:1.227
2023-05-17 15:29:04,407 - INFO - Info:  [192/495 (38%)]	  clean_RMSE: 0.5346 clean_Recall: 20.6186 clean_MAP: 0.0620 clean_RCR: 0.8615   adv_RMSE: 0.5354 adv_Recall: 19.5876 adv_MAP: 0.0595 adv_RCR: 0.8629 time:1.204
2023-05-17 15:29:06,808 - INFO - Info:  [256/495 (50%)]	  clean_RMSE: 0.5212 clean_Recall: 16.4948 clean_MAP: 0.1009 clean_RCR: 0.8825   adv_RMSE: 0.5206 adv_Recall: 15.4639 adv_MAP: 0.0652 adv_RCR: 0.9044 time:1.206
2023-05-17 15:29:09,282 - INFO - Info:  [320/495 (62%)]	  clean_RMSE: 0.5701 clean_Recall: 18.2540 clean_MAP: 0.0790 clean_RCR: 0.859

2023-05-17 15:29:17,0,Chicago,Gsnet,random,min_impr,10,32,0.5417841608881183,20.63208109719738,0.08756782175523166,0.8295174198583164,0.5430782623469452,19.97614788312463,0.08473945656449267,0.8354451130791573,0.05373331885362462,98.97738609910209,0.9879364039248545,0.0026089471413635545


2023-05-17 15:29:21,423 - INFO - Info:  [64/1080 (12%)]	  clean_RMSE: 0.5409 clean_Recall: 20.9524 clean_MAP: 0.0687 clean_RCR: 0.8350   adv_RMSE: 0.5419 adv_Recall: 22.8571 adv_MAP: 0.0776 adv_RCR: 0.8253 time:1.275
2023-05-17 15:29:24,115 - INFO - Info:  [128/1080 (25%)]	  clean_RMSE: 0.5505 clean_Recall: 17.7083 clean_MAP: 0.1596 clean_RCR: 0.8461   adv_RMSE: 0.5498 adv_Recall: 19.7917 adv_MAP: 0.1596 adv_RCR: 0.8318 time:1.333
2023-05-17 15:29:26,793 - INFO - Info:  [192/1080 (38%)]	  clean_RMSE: 0.5346 clean_Recall: 20.6186 clean_MAP: 0.0620 clean_RCR: 0.8615   adv_RMSE: 0.5359 adv_Recall: 18.5567 adv_MAP: 0.0602 adv_RCR: 0.8719 time:1.321
2023-05-17 15:29:29,407 - INFO - Info:  [256/1080 (50%)]	  clean_RMSE: 0.5212 clean_Recall: 16.4948 clean_MAP: 0.1009 clean_RCR: 0.8825   adv_RMSE: 0.5196 adv_Recall: 15.4639 adv_MAP: 0.0687 adv_RCR: 0.9045 time:1.316
2023-05-17 15:29:32,093 - INFO - Info:  [320/1080 (62%)]	  clean_RMSE: 0.5701 clean_Recall: 18.2540 clean_MAP: 0.0790 clean_RCR: 

2023-05-17 15:29:40,0,Chicago,Gsnet,random,pgd_impr,10,32,0.5417841608881183,20.63208109719738,0.08756782175523166,0.8295174198583164,0.5428889634001431,20.63208109719738,0.0853189068960142,0.8255467226594557,0.05144042074326109,98.96075823079481,0.9876630882093521,0.00264228023787187


KeyboardInterrupt: 